# Learning Goals

In Assignment 1, we studied how information is represented by a single spiking neuron. In this assignment, you will learn how to construct networks of spiking neurons for a given cognitive task, how to propagate information through a network, and understanding the intuition behind network design choices. 

Let's first import all the libraries required for this assignment

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Question 1: From single neuron to network of neurons
## 1a.
What computational advantages do networks of neurons offer when compared against information processing by a single neuron? In other words, why do we need networks of neurons? 

## Answer 1a.
> Network of neurons helps us to capture more information about the input and also keep computational overhead in check which otherwise overshoots when we use single neuron for more timesteps.

## 1b. 
Describe the algorithm for the information flow through a network of spiking leaky-integrate-and-fire (LIF) neurons. Specifically, trace out the steps required to compute network output from a given (continuous-valued) inputs. The algorithm should describe how continuous-valued inputs are fed to the SNN input layer, how the layer activations are computed, and how the output layer activity is decoded. Also, provide a diagrammatic overview of the algorithm to aid your explanation. You are free to assume any network size, and input and output dimensions. 

## Answer 1b.
1. Encoding Input
    - Input Image is initially encoded using one of the encoding methods, like rate encoding or temporal encoding.
    - Initially we generate a random image of same dimensions as input image for specified number of timesteps. This random image is compared with original input image pixels and converted to spikes [1] whenever image pixels values are greater tha random image pixel, and no spikes [0] otherwise.
2. Feed Forward Propagation
    - Using the input spikes we calculate the current for next layer: 
        - Compute the current for the hidden layers using layer weights: psp = weight * input
    - Integrate the current (psp) into membrane voltage to calculate spikes for next layer: 
        - v[t] = dv.v[t-1] + psp[t]
    - use thresholding to convert voltage to spikes: 
        - if v[t] >= vth: 
            - generate spikes for neuron
            - Reset v to 0
    - accumulate output spikes until specified number of timesteps
3. Decode Output using arithmetic operations and argmax of python numpy library


<img src="SNN.jpeg" width=700 height=550 />

# Question 2: Elements of Constructing Feedforward Networks
In this exercise, you will implement the two fundamental components of a feedforward spiking neural network: i) layers of neurons and ii) connections between those layers
## 2a. 
As the first step towards creating an SNN, we will create a class that defines a layer of LIF neurons. The layer object creates a collection of LIF neurons and applies input current to it (also called psp_input for postsynaptic input) to produce the collective spiking output of the layer. 

Below is the class definition for a layer of LIFNeurons. Fill in the components to define the layer. 

In [2]:
class LIFNeurons:
    """ Define Leaky Integrate-and-Fire Neuron Layer """

    def __init__(self, dimension, vdecay, vth):
        """
        Args:
            dimension (int): Number of LIF neurons in the layer
            vdecay (float): voltage decay of LIF neurons
            vth (float): voltage threshold of LIF neurons
        
        This function is complete. You do not need to do anything here.
        """
        self.dimension = dimension
        self.vdecay = vdecay
        self.vth = vth

        # Initialize LIF neuron states
        self.volt = np.zeros(self.dimension)
        self.spike = np.zeros(self.dimension)
    
    def __call__(self, psp_input):
        """
        Args:
            psp_input (ndarray): synaptic input current at a single timestep. The shape of this is same as the number of neurons in the layer. 
        Return:
            self.spike: output spikes from the layer. The shape of this should be the same as the number of neurons in the layer. 
        
        Write the expressions for updating the voltage and generating the spikes for the layer given psp_input at one timestep. 
        """
        #Update the voltage
        self.volt = (self.vdecay*self.volt) + psp_input
        
        #Generate the spikes from the voltage 
        self.spike = (self.volt >= self.vth).astype(int)
        
        #Reset the voltage if the neuron spikes
        self.volt[self.volt >= self.vth] = 0
        
        return self.spike

To verify the correctness of your class implementation, create a layer of neurons using the class definition above, and pass through it random inputs. 

In [3]:
#Create a layer of neurons using the class definition above. You can pick any parameter values for the neurons.
dimension = 10
vdecay = 0.5
vth = 0.5
layer = LIFNeurons(dimension, vdecay, vth)

#Create random input spikes with any probability and print them. Numpy random.choice function might be useful here. 
p = 0.6
in_spikes = np.random.choice([0,1], dimension, True, [1-p, p])
print("Input spike: {}".format(in_spikes))

#Propagate the random input spikes through the layer and print the output
out_spikes = layer(in_spikes)
print("Output spike: {}".format(out_spikes))

Input spike: [0 1 1 1 1 1 0 0 1 1]
Output spike: [0 1 1 1 1 1 0 0 1 1]


## 2b.
Now, we will create a class the defines the connection between a presynaptic layer and a postsynaptic layer. To create the connection, we need the activity of the presynaptic layer (also called presynaptic layer activation) and the weight matrix connecting the presynaptic and postsynaptic neurons. The output of the class should be the current for the postsynaptic layer. 

Below is the class definition for Connections. Fill in the components to create the connections. 

In [4]:
class Connections:
    """ Define connections between spiking neuron layers """

    def __init__(self, weights, pre_dimension, post_dimension):
        """
        Args:
            pre_dimension (int): number of neurons in the presynaptic layer
            post_dimension (int): number of neurons in the postsynaptic layer
            weights (ndarray): connection weights of shape post_dimension x pre_dimension

        This function is complete. You do not need to do anything here.

        """
        self.weights = weights
        self.pre_dimension = pre_dimension
        self.post_dimension = post_dimension
    
    def __call__(self, spike_input):
        """
        Args:
            spike_input (ndarray): spikes generated by the pre-synaptic neurons
        Return:
            psp: current for the post-synaptic neurons
        
        Write the operation for computing psp
        """
        
        #Compute psp given spike_input and self.weights
        
        psp = spike_input @ np.transpose(self.weights)
        
        return psp

To verify the correctness of your class implementation, create a connection object and compute the postsynaptic current for random presynaptic activation inputs and random connection weights. You can pick arbitrary values for class arguments. 

In [5]:
#Define the dimensions of the presynaptic layer in a variable
pre_dim = 25

#Define the dimensions of the postsynaptic layer in a variable
post_dim = 8

#Create random presynaptic inputs with any probability. Numpy random choice function might be useful here. 
presynaptic_current = np.random.choice([0,1], pre_dim, True, [1-p, p])  #p = 0.6
# print("presynaptic current: {}".format(presynaptic_current))

#Create a random connection weight matrix. Numpy random rand function might be useful here. 
w_0 = np.random.rand(post_dim, pre_dim)
# print("weight at input layer, w_0: {}".format(w_0))

#Initialize a connection object using the Connection class definition and pass the variables created above as arguments
connect = Connections(w_0, pre_dim, post_dim)

#Compute the current for the postsynaptic layer when the connection object is fed random presynaptic activation inputs
postsynaptic_current = connect(presynaptic_current)

#Print the shape of the current
print("Postsynaptic current shape: {}".format(postsynaptic_current.shape))
# print("postsynaptic current: {}".format(postsynaptic_current))


Postsynaptic current shape: (8,)


# Question 3: Constructing Feedforward SNN
Now that you have implemented the basic elements of an SNN- layer and connection, you are all set to implement a fully functioning SNN. The SNN that you will implement here consists of an input layer, a hidden layer, and an output layer. 

Below is the class definition of an SNN. Your task is to create the layers and connections that form the network using the class definitions in Question 2. Then complete the function to propagate a given input through the network and decode network output. 

In [6]:
class SNN:
    """ Define a Spiking Neural Network with One Hidden Layer """

    def __init__(self, input_2_hidden_weight, hidden_2_output_weight, 
                 input_dimension=784, hidden_dimension=256, output_dimension=10,
                 vdecay=0.5, vth=0.5, snn_timestep=20):
        """
        Args:
            input_2_hidden_weight (ndarray): weights for connection between input and hidden layer. dimension should be hidden_dimension x input_dimension. 
            hidden_2_output_weight (ndarray): weights for connection between hidden and output layer. dimension should be output dimension x hidden dimension. 
            input_dimension (int): number of neurons in the input layer
            hidden_dimension (int): number of neurons in the hidden layer
            output_dimension (int): number of neurons in the output layer
            vdecay (float): voltage decay of LIF neurons
            vth (float): voltage threshold of LIF neurons
            snn_timestep (int): number of timesteps for simulating the network (also called inference timesteps)
        """
        self.snn_timestep = snn_timestep
        
        #Create the hidden layer
        self.hidden_layer = LIFNeurons(hidden_dimension, vdecay, vth)
        
        #Create the output layer
        self.output_layer = LIFNeurons(output_dimension, vdecay, vth)
        
        #Create the connection between input and hidden layer
        self.input_2_hidden = Connections(input_2_hidden_weight, input_dimension, hidden_dimension)
        
        #Create the connection between hidden and output layer
        self.hidden_2_output = Connections(hidden_2_output_weight, hidden_dimension, output_dimension)
        
    
    def __call__(self, spike_encoding):
        """
        Args:
            spike_encoding (ndarray): spike encoding of input
        Return:
            output: decoded output from the network
        """
        
        #Initialize an array to store the decoded network output for all neurons in the output layer
        spike_output = np.zeros(self.output_layer.dimension)
        
        #Loop through the simulation timesteps and process the input at each timestep tt
        for tt in range(self.snn_timestep):
            #Propagate the input through the input to hidden layer and compute current for hidden layer
            hidden_layer_current = self.input_2_hidden(spike_encoding[tt])
            # print("hidden_layer_current at tt:{}\n{}".format(tt, hidden_layer_current))
           
            #Compute hidden layer spikes 
            hidden_layer_spikes = self.hidden_layer(hidden_layer_current)
            # print("hidden_layer_spikes at tt:{}\n{}".format(tt, hidden_layer_spikes))

            #Propagate hidden layer inputs to output layer and compute current for output layer
            output_layer_current = self.hidden_2_output(hidden_layer_spikes)
            # print("output_layer_current at tt:{}\n{}".format(tt, output_layer_current))
            
            #compute output layer spikes
            output_layer_spikes = self.output_layer(output_layer_current)
            # print("output_layer_spikes at tt:{}\n{}".format(tt, output_layer_spikes))
            
            #Decode spike outputs by summing them up
            spike_output = spike_output + output_layer_spikes
            # print("spike_output at tt:{}\n{}".format(tt, spike_output))
            
        return spike_output

To verify the correctness of your class implementation, define the arguments to initialize the SNN. Then initialize the SNN and pass through it random inputs and compute network outputs. You can pick arbitrary values for class arguments. 

In [7]:
#Define the dimensions of the input layer in a variable
input_dimension = 9

#Define the dimensions of the hidden layer in a variable
hidden_dimension = 4

#Define the dimensions of the output layer in a variable
output_dimension = 2

#Define vdecay in a variable
vdecay = 0.5

#Define vth in a variable
vth = 0.5

#Define snn_timesteps in a variable
snn_timestep = 3

#Create random input to hidden layer weights. Numpy random rand function might be useful here
input_2_hidden_weight = np.random.rand(hidden_dimension, input_dimension)

#Create random hidden to output layer weights. Numpy random rand function might be useful here
hidden_2_output_weight = np.random.rand(output_dimension, hidden_dimension)

#Create random spike inputs to the network. Numpy random choice function might be useful here
spike_encoding = np.random.choice([0,1], (snn_timestep, input_dimension), True, [1-p, p]) # considering 3 timesteps - dimensions 3 x input dimension

#Print the inputs
print("Spike Input:\n {}".format(spike_encoding))

#Create an SNN object using the class definition and variables defined above
snn = SNN(input_2_hidden_weight, hidden_2_output_weight, input_dimension, hidden_dimension, output_dimension, vdecay, vth, snn_timestep)

#Pass the random spike inputs through the SNN and print the output of the SNN
out = snn(spike_encoding)
print("Out: {}".format(out))


Spike Input:
 [[0 1 0 0 1 0 1 1 1]
 [1 1 1 1 0 0 1 1 1]
 [1 1 1 1 1 0 0 1 1]]
Out: [3. 3.]


# Question 4: SNN for Classification of Digits
So far we have learnt how to construct SNNs for random inputs. In this exercise, you will use your implementation of SNNs to classify real-world data, taking the dataset of handwritten digits as an example. The dataset is provided as numpy arrays in the folder "data". Each sample in the MNIST dataset is a 28x28 image of a digit and a label (between 0 and 9) of that image. We will be dealing with batches, which means that we will read a fixed number of samples from the dataset (also called the batch size).

## 4a. 
First, we need to write two helper functions- to read the data from the saved data files, and to convert an image into spikes. The function to read the data is already written for you. You need to complete the function for encoding the data into spikes. 

In [8]:
def read_numpy_mnist_data(save_root, num_sample):
    """
    Read saved numpy MNIST data
    Args:
        save_root (str): path to the folder where the MNIST data is saved
        num_sample (int): number of samples to read
    Returns:
        image_list: list of MNIST image
        label_list: list of corresponding labels
    
    This function is complete. You do not need to do anything here.
    """
    image_list = np.zeros((num_sample, 28, 28))
    label_list = []
    for ii in range(num_sample):
        image_label = pickle.load(open(save_root + '/' + str(ii) + '.p', 'rb'))
        image_list[ii] = image_label[0]
        label_list.append(image_label[1])

    return image_list, label_list

def img_2_event_img(image, snn_timestep=20):
    """
    Transform image to spikes, also called an event image
    Args:
        image (ndarray): image of shape batch_size x 28 x 28
        snn_timestep (int): spike timestep
    Returns:
        event_image: event image- spike encoding of the image
        
    Complete the expression for converting the image to spikes (event image)
    """
    
    #Reshape the image. Do not touch this code
    batch_size = image.shape[0]
    image_size = image.shape[2]
    image = image.reshape(batch_size, image_size, image_size, 1)
    
    #Generate a random image of the shape batch_size x image_size x image_size x snn_timestep. Numpy random rand function will be useful here. 
    ## rand_img = np.random.rand(snn_timestep, batch_size, image_size*image_size)
    rand_img = np.random.rand(batch_size, snn_timestep, image_size*image_size)

    #Generate the event image
    img = image.reshape(batch_size, image_size*image_size)
    ## img = np.repeat(img[np.newaxis, :, :], snn_timestep, axis=0)
    img = np.repeat(img[:, np.newaxis, :], snn_timestep, axis=1)
    event_image = (img > rand_img).astype(int)

    return event_image

To verify the correctness of your class implementation, load a sample digit from the saved file and convert it into an event image. Then print the shape of the event image. 

In [10]:
#Load 1000 samples from the MNIST dataset using the read function defined above
num_sample = 1000
save_root = "data/mnist_test"
X, y = read_numpy_mnist_data(save_root, num_sample)

#Print the shape of the data
print("Input Data Shape: [batch_size x img_width x img_height] {}".format(X.shape))

#Convert the images to event images
event_img = img_2_event_img(X)

#Print the shape of the event image
print("Event Image Shape: [batch_size x timestep x flattened_image] {}".format(event_img.shape))


Input Data Shape: [batch_size x img_width x img_height] (1000, 28, 28)
Event Image Shape: [batch_size x timestep x flattened_image] (1000, 20, 784)


## 4b. 
Next, we need another helper function to compute the classification accuracy of the network. The classification accuracy is defined as the percentage of the samples that the network classifies correctly. To compute the classification accuracy, you need to:

- Propagate each input through the network and obtain the network output.
- Based on the network output, the class of the image is the one for which the output neuron has maximum value. Let's call this predicted class. 
- Compare the predicted class against the true class. 
- Compute accuracy as the percentage of correct predictions. 

Below is the function for computing the test accuracy. The function takes in as arguments the SNN, directory in which the MNIST data is saved, and the number of samples to take from the MNIST dataset. Your task is to use the helper functions created above to load the data, convert into event images, and then compute network prediction and accuracies. 

In [12]:
def test_snn_with_mnist(network, data_save_dir, data_sample_num):
    """
    Test SNN with MNIST test data
    Args:
        network (SNN): defined SNN network
        data_save_dir (str): directory for the test data
        data_sample_num (int): number of test data examples
    """
    #Read image and labels using the read function
    test_image_list, test_label_list = read_numpy_mnist_data(data_save_dir, data_sample_num)
    
    #Convert the images to event images
    event_image = img_2_event_img(test_image_list)
    
    ## out = network(event_image)
    ## predictions = np.argmax(out, axis=1)

    #Initialize number of correct predictions to 0
    correct_prediction = 0
    ## correct_prediction += sum((predictions == test_label_list).astype(int))

    #Loop through the test images
    for ii in range(data_sample_num):
        #Compute network output for each image. You might have to reshape the image using Numpy reshape function so that its appropriate for the SNN
        out = network(event_image[ii])
        
        #Determine the class of the image from the network output. Numpy argmax function might be useful here
        predictions = np.argmax(out, axis=0)
        
        #Compare the predicted class against true class and update correct_prediction counter
        if predictions == test_label_list[ii]:
            correct_prediction += 1
        
    #Compute test accuracy
    
    test_accuracy = correct_prediction/data_sample_num
    print("{}/{} = {}".format(correct_prediction,len(test_label_list),test_accuracy))
    # for i in range(len(predictions)):
    #     print("predicted = {} | true label = {}".format(predictions[i], test_label_list[i]))

    return test_accuracy

# Question 5: Tuning Membrane Properties for Correct Classification 
Great! We have everything that we need to measure the performance of the SNN for classification of MNIST digits. For this, we first need to create the SNN using the class definition we wrote in Q.3. Then we need to call the test function that we wrote in Q.4b. However, note that the SNN needs the connection weights between the layers as inputs. These weights are typically obtained as a result of "training" the network for a given task (such as MNIST classification). However, since training the network isn't a part of this assignment, we provide to you already trained weights. 

## 5a. 
Your task in this exercise is to initialize an SNN with vdecay=1.0 and vth=0.5. Test the SNN on MNIST dataset and obtain the classification accuracy.  

In [13]:
#Load the weights. Do not touch this code
snn_param_dir = 'save_models/snn_bptt_mnist_train.p'
snn_param_dict = pickle.load(open(snn_param_dir, 'rb'))
input_2_hidden_weight = snn_param_dict['weight1']
hidden_2_output_weight = snn_param_dict['weight2']

#Define a variable for vdecay
vdecay = 1.0

#Define a variable for vth
vth = 0.5

#Define parameters
data_save_dir = "data/mnist_test"
data_sample_num = 1000
input_dimension=784 
hidden_dimension=256 
output_dimension=10
snn_timestep=20

#Create the SNN using the class definition in Q3 and the variables defined above
network = SNN(input_2_hidden_weight, hidden_2_output_weight, input_dimension, hidden_dimension, output_dimension, vdecay, vth, snn_timestep)

#Compute test accuracy for the SNN on 1000 examples from MNIST dataset and print it
test_accuracy = test_snn_with_mnist(network, data_save_dir, data_sample_num)
print("Test Accuracy = {}".format(test_accuracy))


241/1000 = 0.241
Test Accuracy = 0.241


What could be a possible reason for the poor accuracy?

## Answer 5a. 
> As we have fixed our vdecay to 1.0, the voltage does not decay and after a point there is continuous spike for every input which is unable to then differentiate between them and hence we have a poor accuracy. We decrease vdecay to 0.5 or 0.6 and then we will be able to get a much better accuracy.

## 5b. 
Can you tune the membrane properties (vdecay and vth) to obtain higher classification accuracies?

In [14]:
#Write your implementation of Question 5b. here
vdecay = 0.5
vth = 0.5
tuned_snn = SNN(input_2_hidden_weight, hidden_2_output_weight, input_dimension, hidden_dimension, output_dimension, vdecay, vth)
tuned_accuracy = test_snn_with_mnist(tuned_snn, data_save_dir, data_sample_num)
print("Tuned Test Accuracy = {}".format(tuned_accuracy))

970/1000 = 0.97
Tuned Test Accuracy = 0.97


## 5c.
Based on your response to Questions 5a. and 5b., can you explain how membrane properties affect network activity for classification?

## Answer 5c.
> With varying membrane properties, we get a pattern of varying spikes from different neurons. It is this unique combination of different neuron spikes that help us to classify different digits. These neuronal spikes are governed by membrane properties like vdecay and vthreshold. Setting our vthreshold too high or too low can result in very bad spiking behaviour, similarly setting vdecay too high or too low can have same effect.